In [ ]:
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse
import random
from scrawl import *
    
# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5f69d22ef472d6646f577fa6'
site = 'tehrantimes_com'
site_name = "Tehran Times iran's Leading International Day"
c = Crawl()  # creating object

log_path = c.create_directories(project_path, client_id, site)

# create image directories
image_directory = c.create_image_directories(project_path)

# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# initialize variables
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_article_url = 0
publish_source = 'tehrantimes.com'  
country = 'Iran'
language = 'English'
images_path = []

rss_pages = c.download_page('https://www.tehrantimes.com/rss')

for i in rss_pages.split('<item>')[1:]:
    source_link = c.scrap('<link>(.*?)</link>', i)

    source_link_query = {'source_link':source_link}
    dic = cl_data.find_one(source_link_query,{'source_link': 1}) 

    if dic:
        duplicate_data += 1
        continue    
        
    logger.info(f'Fetching {source_link}\n')
    time.sleep(random.randint(1,3))

    page = c.download_page(source_link)

    if page.startswith('Unable to fetch'):
        logger.info(page) # writes error message with error code
        unable_to_fetch_article_url += 1
        continue
    
    
    source_headline = c.scrap('<title>(.*?)</title>', i)
     
    if not source_headline:
        logger.info(f'Skipping due to headline {source_link}\n')
        skipped_due_to_headline += 1
        continue
   
    pub_date, publish_time = '', ''            
            
    try: 
        date_time_str = c.scrap('<pubDate>(.*?)</pubDate>', i)#Tue, 01 Jun 2021 07:00:10
        date_time_str = re.sub('GMT','',date_time_str)
        date_time_str = re.sub('[^\w+]','',date_time_str)
        date_time_obj = datetime.strptime(date_time_str, '%a%d%b%Y%H%M%S')
        ist_date_time = date_time_obj + timedelta(hours = 1,minutes = 0)  
        ist_date_time = ist_date_time.strftime('%Y-%m-%d %H:%M:%S')  
        pub_date = ist_date_time[:10]
        publish_time = ist_date_time[11:]

    except:
        pass
  
    if not pub_date:
        logger.info(f'Skipping due to date {source_link}\n')
        skipped_due_to_date += 1
        continue
    
    if pub_date != created_on:
        break 
    
    
    source_content = c.scrap('<div\s*class="item-text">(.*?)<script\s*type="text/javascript">',page)
    source_content = source_content.replace("<![CDATA[","").replace("]]>","")
    source_content = re.sub('&.*?;','',source_content)
    source_content = c.strip_html(source_content)

    
    if not source_content:
        logger.info(f'Skipping due to content {source_link}\n')
        skipped_due_to_content += 1
        continue
    
    journalist =  ''
    if not journalist: journalist = 'NA'
        
        
  # current date and time 
    harvest_time = datetime.now().strftime("%H:%M:%S")

    # temp link
    temp_link = source_link

    # headline and content 
    headline = source_headline
    content = source_content

    # overall_tonality
    overall_tonality = ''

    # word count
    word_count = len((source_headline + ' ' + source_content).split())

    html_content = ''

    # image_urls
    image_urls = []

   # storing the above data in a dictionary
    clientdata ={
                    "client_master" : client_id, 
                    "articleid":client_id,
                    "medium":'Web' ,
                    "searchkeyword":[],
                    "entityname" : [] ,
                    "process_flage":"1",
                    "na_flage":"0",
                    "na_reason":"",
                    "qc_by":"",
                    "qc_on":"",
                    "location":"",
                    "spokeperson":"",
                    "quota":"",
                    "overall_topics":"",
                    "person":"",
                    "overall_entites":"",
                    "overall_tonality": overall_tonality,
                    "overall_wordcount":word_count,
                    "article_subjectivity":"",
                    "article_summary":"",
                    "pub_date":pub_date,
                    "publish_time":publish_time,
                    "harvest_time":harvest_time,
                    "temp_link":temp_link,
                    "publish_source": publish_source,
                    "programme":'null',
                    "feed_class":"News",
                    "publishing_platform":"",
                    "klout_score":"",
                    "journalist":journalist,
                    "headline":headline,
                    "content":content,
                    "source_headline":source_headline,
                    "source_content":source_content,
                    "language":language,
                    "presence":'null',
                    "clip_type":'null',
                    "prog_slot":'null',
                    "op_ed":'0',
                    "location_mention":'',
                    "source_link":source_link,
                    "author_contact":'',
                    "author_emailid":'',
                    "author_url":'',
                    "city":'',
                    "state":'',
                    "country":country,
                    "source":publish_source,
                    "foot_fall":'',
                    "created_on":created_on,
                    "active":'1',
                    'crawl_flag':2,
                    "images_path":images_path,
                    "html_content":html_content
                } 

    cl_data.insert_one(clientdata)  
    no_of_data += 1


logger.info('Iteration complete\n')   
logger.info(f'Number of data: {no_of_data}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Unable to fetch article url: {unable_to_fetch_article_url}\n')
logger.info(f'Skipped due to headline: {skipped_due_to_headline}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Skipped due to date: {skipped_due_to_date}\n')
logger.info(f'country: {country}\n')
logger.info(f'language: {language}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n')

https://www.tehrantimes.com/news/461661/Iran-victorious-over-Italy-in-2021-VNL
Iran victorious over Italy in 2021 VNL
2021-06-04
TEHRAN – Iran earned their third successive win in the 2021 Volleyball Nations League, beating Italy 3-1 (26-24, 29-27, 21-25, 25-22) on Friday. 

  Saber Kazemi notched a match-high 24 points for Iran and Italy’s Gabriele Nelli scored 20 points. 
  Iran lost to Japan and Russia in their first two matches but defeated the Netherlands and Canada in the upcoming matches. 
  Iran will meet Bulgaria Saturday night in Week Two’s last match. 
  A total of 32 of the world’s top national teams compete at the VNL 2021 bubble in Rimini, Italy to showcase electrifying volleyball action for millions of fans worldwide. 
  The 16-team tournament began with a round-robin phase where each side play 15 games.
https://www.tehrantimes.com/news/461660/Iran-stronger-than-ever-Leader
Iran stronger than ever: Leader
2021-06-04
TEHRAN - In a live televised speech on the anniversary 

Senior Iranian diplomat elaborates on Salehi’s letter to IAEA chief
2021-06-02
TEHRAN - Iran’s envoy to the International Atomic Energy Agency, Kazem Gharibabadi, has given more details about the letter that Iran’s nuclear chief sent to IAEA Director-General Rafael Grossi.

  The letter, sent via Gharibabadi, contained details about Iran’s decision to announce the expiration of a technical understanding between Iran and the IAEA. 
  The letter was announced after the Agency released a quarterly report on Iran in which it accused Iran of failing to explain traces of uranium found at several allegedly undeclared sites. The letter also came after Grossi announced on May 24 that he had agreed with Iran to extend by one month a February deal between Iran and the IAEA allowing the UN nuclear watchdog to continue necessary monitoring activities. 
  However, Gharibabadi said on Twitter that Ali Akbar Salehi, head of the Atomic Energy Organization of Iran (AEOI), in his letter, told Grossi that

Leader’s support for Guardian Council ‘heartens us’: Jannati
2021-06-02
TEHRAN - Guardian Council secretary Ayatollah Ahmad Jannati on Wednesday thanked Leader of the Islamic Revolution Ayatollah Ali Khamenei for backing the Guardian Council, saying the Leader’s support has heartened the members of the vetting body.

  “It is necessary to announce my thanks and praise and those of the members of the Guardian Council to the recent speech by the Leader. He has always backed the position and performance of the Guardian Council in different times and this has heartened us,” Jannati told a meeting of the council. 
  The Guardian Council confirmed seven candidates to run for the presidential post. The names of the confirmed candidates were announced by the Interior Ministry on May 25. 
  In a speech to the parliament through videoconference on May 28,the Leader said, “The honorable Guardian Council did what it deemed necessary according to its responsibility.” Ayatollah Khamenei also thanked

$3.3m paid to support tourism businesses in Hamedan
2021-06-02
TEHRAN – The Iranian government has paid some 140 billion rials (some $3.3 million at the official exchange rate of 42,000 rials per dollar) in loans to the tourism businesses affected by the coronavirus pandemic in Hamedan province.

  Throughout the west-central province, 170 units, engaged in the tourism industry, have received the loans, the deputy provincial tourism chief has said. 
  Tourism and handicrafts in Hamedan employ more than 50,000 people directly, and both sectors are heavily affected by the outbreak of coronavirus and require the government’s assistance, Ali Khaksar said on Tuesday. 
  Back in January, Hamedan province’s tourism chief Ali Malmir announced that the tourism industry of the province has taken some 2.48 trillion rials ($59 million) hit from the coronavirus outbreak over the past couple of months. 
  Accommodation centers, hotels, restaurants, and halls have suffered major damages during the ti